In [1]:
# try to create the U-Net architecture 
# in Tensorflow's Keras API
# to remove foreground from cosmological simulations
# by Lachlan Lancaster

from __future__ import absolute_import, division, print_function
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

/Users/lachlanl/anaconda/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.12.0


In [2]:
inputs = keras.layers.Input(shape=(64,64,30),name="image_input")
conv1a = keras.layers.Conv2D(64,7,activation=tf.nn.relu)(inputs)
conv1b = keras.layers.Conv2D(64,7,activation=tf.nn.relu)(conv1a)
mp1 = keras.layers.MaxPool2D(2)(conv1b)
conv2a = keras.layers.Conv2D(128,3,activation=tf.nn.relu)(mp1)
conv2b = keras.layers.Conv2D(128,3,activation=tf.nn.relu)(conv2a)
mp2 = keras.layers.MaxPool2D(2)(conv2b)
conv3a = keras.layers.Conv2D(256,3,activation=tf.nn.relu)(mp2)
conv3b = keras.layers.Conv2D(256,2,activation=tf.nn.relu)(conv3a)
mp3 = keras.layers.MaxPool2D(2)(conv3b)
# symmetric upsampling path
upsamp2 = keras.layers.UpSampling2D(5)(mp3)
upconv2 = keras.layers.Conv2DTranspose(128,3)(upsamp2)
concat2 = keras.layers.concatenate([conv2b,upconv2])
conv4a =  keras.layers.Conv2D(128,3,activation=tf.nn.relu)(concat2)
conv4b =  keras.layers.Conv2D(128,3,activation=tf.nn.relu)(conv4a)
upsamp1 = keras.layers.UpSampling2D(3)(conv4b)
conv5 =  keras.layers.Conv2D(64,3,activation=tf.nn.relu)(upsamp1)
concat1 = keras.layers.concatenate([conv1b,conv5])
upconv1a = keras.layers.Conv2DTranspose(64,7)(concat1)
upconv1b = keras.layers.Conv2DTranspose(64,7)(upconv1a)

output = keras.layers.Conv2DTranspose(30,7)(upconv1a)
#output = keras.layers.Conv2DTranspose(30,17)(conv3b)

model = keras.models.Model(inputs=inputs,outputs=output)

In [3]:
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image_input (InputLayer)        (None, 64, 64, 30)   0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 58, 58, 64)   94144       image_input[0][0]                
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 52, 52, 64)   200768      conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 26, 26, 64)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (

In [4]:
data = np.load("/Volumes/My Passport for Mac/lachlanl/21cm_project/sims/observed_nsim100_fg1.npy")

In [5]:
signal = np.load("/Volumes/My Passport for Mac/lachlanl/21cm_project/sims/cosmo_nsim100_fg1.npy")

In [6]:
print (data.shape,signal.shape)

(19200, 64, 64, 30) (19200, 64, 64, 30)


In [8]:
x_train = data[:18000]
y_train = signal[:18000]
x_val = data[19000:-200]
y_val = signal[19000:-200]
x_test = data[-200:]
y_test = signal[-200:]

In [10]:
history = model.fit(x_train,y_train,batch_size=64,epochs=3,validation_data=(x_val, y_val))

Train on 18000 samples, validate on 0 samples
Epoch 1/3
 1920/18000 [==>...........................] - ETA: 47:47 - loss: 3296217552.4012 - acc: 0.0419

KeyboardInterrupt: 